## Load In Libraries

In [1]:
import math
import ctREFPROP.ctREFPROP as ct
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Hack for my installation
os.environ['RPPREFIX'] = r'C:\\Program Files (x86)\REFPROP'

root = os.environ['RPPREFIX']
#r = ct.REFPROPFunctionLibrary(os.path.join(root, 'refprop.dll'))
r = ct.REFPROPFunctionLibrary(os.path.join(root, 'REFPRP64.dll'))
r.SETPATHdll(root)

from numpy import loadtxt
#from multiprocessing import Pool

## Real Gas Data Generation

In [5]:
fuel_type='hexane.FLD'
amb_type='nitrogen.FLD'

## Conditions
dP = 200
Psweep = np.arange(100,14000+dP,dP)
dT = 2
Tsweep = np.arange(380,651+dT,dT) # 500 for butane and below 600 and above
Tcrit = 507.82
Tambsweep = Tcrit*np.array([0.7])
dX=0.01
xf = np.arange(0,1+dX,dX)

mix_list=fuel_type+"|"+amb_type
ierr, herr = r.GERG08dll(2,0)
if ierr !=0:
    raise ValueError(str(ierr) + ':' + herr)

r.FLAGSdll('GERG',1)
ierr, herr = r.SETUPdll(2,mix_list,'HMX.BNC','DEF')

if ierr !=0:
    raise ValueError(str(ierr) + ':' + herr)
zf=[1,0]
zamb=[0,1]

MWf = r.WMOLdll(zf)
MWamb = r.WMOLdll(zamb)



totalsteps = Tambsweep.size*Psweep.size

######################################################################################
Tmix = np.zeros((((Psweep.size,Tsweep.size,Tambsweep.size,xf.size))))




def run_complex_operations(operation, input):
    #pool.map(operation, input)
    for i in input:
        operation(i)       
input = range(xf.size)
#processes_count = 8
           
counter = -1
ww = -1
for Tamb in Tambsweep:
    ww+=1

    


    jj = -1
    for Pamb in Psweep:
        jj+=1
        counter +=1
        HAmb =  r.TPFLSHdll(Tamb,Pamb,zamb).h
        
        kk = -1
        for TfuelIn in Tsweep:
            kk+=1
            ## Inlet Conditions

            HfInlet =  r.TPFLSHdll(TfuelIn,Pamb,zf).h
            
            def complex_operation(input_index):
                Tmix[jj,kk,ww,input_index] = r.PHFLSHdll(Pamb,HfInlet*xf[input_index]+HAmb*(1-xf[input_index]),[xf[input_index],1-xf[input_index]]).T
                if  r.PHFLSHdll(Pamb,HfInlet*xf[input_index]+HAmb*(1-xf[input_index]),[xf[input_index],1-xf[input_index]]).ierr!=0:
                    Tmix[jj,kk,ww,input_index] = None 
                    
            input = range(xf.size)
            run_complex_operations(complex_operation,input)
            #if __name__ == '__main__':
             #   processes_pool = Pool(processes_count)
             #   run_complex_operations(complex_operation, range(xf.size), processes_pool)
                
                
        print('%d / %d'%(counter, totalsteps),end='\r')
     





In [6]:
TSatV = np.zeros((((Psweep.size,Tsweep.size,Tambsweep.size,xf.size))))
TSatL = np.zeros((((Psweep.size,Tsweep.size,Tambsweep.size,xf.size))))


def run_complex_operations(operation, input):
    #pool.map(operation, input)
    for i in input:
        operation(i)       
input = range(xf.size)
#processes_count = 8

counter = -1
jj = -1
for Pamb in Psweep:
    jj+=1
    counter +=1
    
    def complex_operation(input_index):
        TSatV[jj,:,:,input_index] = r.SATPdll(Pamb,[xf[input_index],1-xf[input_index]],2).T
        if r.SATPdll(Pamb,[xf[input_index],1-xf[input_index]],2).ierr!=0:
            TSatV[jj,:,:,input_index] = None
        TSatL[jj,:,:,input_index] = r.SATPdll(Pamb,[xf[input_index],1-xf[input_index]],1).T
        if r.SATPdll(Pamb,[xf[input_index],1-xf[input_index]],1).ierr!=0:
            TSatL[jj,:,:,input_index] = None 
            input = range(xf.size)
    run_complex_operations(complex_operation,input)
    print('%d / %d'%(counter, Psweep.size),end='\r')
    
IsTwoPhase = Tmix<(TSatV+0.5)
TwoPhaseSum = np.sum(IsTwoPhase, axis=3)
TwoPhaseBin = TwoPhaseSum!=0
TwoPhaseBin = TwoPhaseBin.astype('int64') # implicitly changes all the "False" values to 0
Tline = np.zeros((Psweep.size,Tambsweep.size))
for j in list(range(Tambsweep.size)):
    for i in list(range(Psweep.size)):
        try:
            loc = np.max(np.nonzero(TwoPhaseBin[i,:,j]))
            Tline[i,j] = Tsweep[loc+1]
        except Exception:
            pass
            Tline[i,j] = None


array([[ nan],
       [382.],
       [404.],
       [420.],
       [434.],
       [446.],
       [454.],
       [464.],
       [470.],
       [478.],
       [484.],
       [490.],
       [496.],
       [502.],
       [508.],
       [512.],
       [518.],
       [522.],
       [526.],
       [528.],
       [532.],
       [534.],
       [536.],
       [540.],
       [542.],
       [542.],
       [544.],
       [546.],
       [548.],
       [548.],
       [550.],
       [550.],
       [552.],
       [552.],
       [554.],
       [554.],
       [556.],
       [556.],
       [556.],
       [558.],
       [558.],
       [558.],
       [560.],
       [560.],
       [560.],
       [560.],
       [562.],
       [562.],
       [562.],
       [562.],
       [562.],
       [562.],
       [562.],
       [564.],
       [564.],
       [564.],
       [564.],
       [564.],
       [564.],
       [564.],
       [564.],
       [564.],
       [564.],
       [564.],
       [564.],
       [564.],
       [56

In [13]:

import pandas as pd
import numpy as np
from scipy.signal import savgol_filter

Tline1 = Tline[:,0]
a = pd.Series(Tline1)
i=-1
while i<0:
    i+=1
    aint = a.interpolate(method="polynomial", order=1)
    Tline1mod2 = aint.to_numpy()
    Tline1mod3 = savgol_filter(Tline1mod2, 9, 1)
np.count_nonzero(np.isnan(Tline1mod2))
Tline1mod2 = Tline1mod3


In [7]:
Filename = 'Hexane_0,7_data.txt'
PsweepVector = np.ravel(Psweep)
TmixVector = np.ravel(Tline)
f= open(Filename,"w+")
for i in range(Tline.size):
    f.write("%f ," % Tline[i])
    f.write("%f\n" %  PsweepVector[i])
f.close()
print('%d / %d'%(ww, Tline.size-1),end='\r')

